# Avocados Technical Task - EDA and Model

---

In this notebook we explore the data, perform the feature engineering, and test the model that we built in Numpy.

### Quick EDA

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
from loguru import logger
import matplotlib.pyplot as plt

# Set parent folder as root to import local modules
module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)

# Remove default logger and set level to INFO
logger.remove()
logger.add(sys.stderr, level="DEBUG")

from src.base.model import LinearRegression, rmse, r2
from src.base.preprocessing import RepeatingRadialBasisFunction, sin_transformer, cos_transformer

In [ ]:
data_path = os.path.join(module_path, "data")
df = pd.read_csv(os.path.join(data_path, "avocado.csv"))
df.head()

In [ ]:
df.dtypes

The dataset consists of a mix of numerical and categorical variables. It also includes temporal information in the form of date and year. The dates are read as strings so we cast them to pandas datetime objects.

In [ ]:
# Removing useless columns
df.drop(columns=["Unnamed: 0"], inplace=True)

# Converting dates from strings to datetime objects
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

In [ ]:
# Do some checks on the size of the data
logger.info(f"Dataset shape: {df.shape} .")
logger.info(f"""Dataset number unique dates: {df["Date"].nunique()} .""")
logger.info(f"""Dataset number unique regions: {df["region"].nunique()} .""")
logger.info(f"""Dataset number unique types: {df["type"].nunique()} .""")
logger.info(f"Dataset unique values: {df.drop_duplicates().shape} .")
logger.info(
    f"""Dataset min date: {df["Date"].min().date()} """
    f"""and max date: {df["Date"].max().date()} ."""
)

The `Total Bags` column seems to be a linear combination of the other columns that are related to the number of bags. If this is the case, we must drop this column otherwise the collinearity might cause issues with the linear model.

In [ ]:
df["Delta"] = np.abs(df["Total Bags"] - (df["Small Bags"] + df["Large Bags"] + df["XLarge Bags"]))
max_delta = df["Delta"].max()
logger.info(
    f"""Largest difference between column `Total Bags` and sum of columns """
    f"""`Small Bags`, `Large Bags`, and `XLarge Bags`: {max_delta:,.2f} ."""
)
df.drop(columns=["Total Bags"], inplace=True)

In [ ]:
try:
    cart_prod = df["Date"].nunique() * df["type"].nunique() * df["region"].nunique()
    assert(df.shape[0] == cart_prod)
except AssertionError:
    logger.warning(
        f"""The lenght of the dataframe ({(df.shape[0]):,.0f}) is different """
        f"""from the that of the cartesian product of `Date`, `type`, and `region` """
        f"""({cart_prod:,.0f}) ."""
    )

In [ ]:
print(df["region"].unique())

In [ ]:
df_test = df[
    (df["Date"] == pd.Timestamp("2015-12-27")) & 
    (df["type"] == "conventional")
]
logger.info(f"""Number of regions in the sample dataset: {df_test["region"].nunique()} .""")
logger.info(
    f"""The combined toal volume across all regions apart from TotalUS is: """
    f"""{df_test[df_test["region"] != "TotalUS"]["Total Volume"].sum():,.2f} ."""
)
logger.info(
    f"""The total volumefor region TotalUS is: """
    f"""{df_test[df_test["region"] == "TotalUS"]["Total Volume"].sum():,.2f} ."""
    )

Finally we check for any missing date values and inconsistencies between the date and the year. We also check whether the `year` has any impact on the price.

In [ ]:
# Check for any missing date
sorted_dates = sorted(df["Date"].unique())
for i in range(df["Date"].nunique() - 1):
    dt_c, dt_n = sorted_dates[i], sorted_dates[i + 1]
    if dt_c + pd.Timedelta(days=7) != dt_n:
        logger.error(f"Error: date missing {format(dt_c + pd.Timedelta(days=7))} .")

# Check for any inconsistencies between date and year
if (df["Date"].dt.year == df["year"]).min() is False:
    logger.error("Error: Dates and years not matching.")

In [ ]:
years = sorted(df["year"].unique())

avgs = df.groupby("year").agg({"AveragePrice": "mean"}).to_dict()["AveragePrice"]

fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax.boxplot([df[df["year"] == y]["AveragePrice"].values for y in years])
ax.scatter(y=[avgs[y] for y in years], x=np.arange(1, len(years) + 1), marker='*')

ax.set_xlabel("Year")
ax.set_ylabel("Avocados Prices")
ax.set_xticklabels(years)
fig.suptitle("Avocados prices statistics by year")
plt.show()

### Feature Engineering

Here we perform the feature engineering. 

- `AveragePrice` will be our target variable
- For the numerical features, we transform them by subtracting the mean and dividing by the standard deviation
- For the categorical features, we transform them to binary values using One Hot encoding:
     *  `type` has only two values so it's fine to just encode it as binary
     *  `region` has 54 distinct values so we need to do a bit more processing
     
     <br/>
- For the date features, we use repeating encodings to account for the seasonality of month and week of the year. As we observed earlier the year does not play a big factor in this dataset so we decide to drop it
     - We transform the month of the year using repeating radial basis functions
     - We transform the week of the year using the sine / cosine transformations

In [ ]:
target_col = "AveragePrice"
num_cols = [c for c in df.columns if df[c].dtype == "float64"]

In [ ]:
# This will be our target variable
y = df[target_col].values
logger.info(f"y shape: {y.shape} .")

# For the numerical volumns, we transform them by subtracting the mean and 
# dividing by the standard deviation
X_num = df[num_cols].values
X_num = (X_num - np.mean(X_num, axis=0)) / np.std(X_num, axis=0)
logger.info(f"X_num shape: {X_num.shape} .")

# For the categorical columns, we transform them to binary values using One Hot 
# encoding. `type` has only two values so it's fine to just encode it as binary, 
# while for `region` we need to do a bit more processing
X_type = df["type"].map(dict(zip(df["type"].unique(), np.arange(2)))).values
X_type = np.expand_dims(X_type, axis=-1)
logger.info(f"X_type shape: {X_type.shape} .")

n_regs = df["region"].nunique() # 54 
# We could have even used 53 as one column is redundant
target_regs = df["region"].map(dict(zip(df["region"].unique(), np.arange(n_regs)))).values
X_region = np.eye(n_regs)[target_regs]
logger.info(f"X_region shape: {X_region.shape} .")

In [ ]:
# For the dates, we choose to encode them in different ways:
# - First as we saw earlier we don't need the year column as 
# - Then we transform the month of the year using repeating radial basis functions
# - Then we transform the week of the year using the sine / cosine transformations
df["DoY"] = df["Date"].dt.day_of_year

In [ ]:
rbf_month = RepeatingRadialBasisFunction(n_periods=12)
rbf_month.fit(X=df["DoY"])
X_month = rbf_month.transform(df["DoY"])
logger.info(f"X_month shape: {X_month.shape} .")

In [ ]:
_ = pd.DataFrame(index=df["Date"].values, data=X_month).plot(
    subplots=True, figsize=(16, 8), sharex=True, legend=False, ylim=(0, 1),
    title="Repeating Radial Basis Functions"
)

In [ ]:
X_sin = sin_transformer(df["DoY"].values, period=52)
X_cos = cos_transformer(df["DoY"].values, period=52)
X_week = np.concatenate([X_sin[:, np.newaxis], X_cos[:, np.newaxis]], axis=1)
logger.info(f"X_week shape: {X_week.shape} .")

In [ ]:
_ = pd.DataFrame(index=df["Date"].values, data=X_week, columns=['sin', 'cos']).plot(
    subplots=False, figsize=(16, 4), legend=True, ylim=(-1, 1),
    title="Sine / Cosine transformation"
)

### Model training and testing

Here we create, train, and test our Numpy Linear Regression model. 

We start by concatenating all the features that we have engineered into a single Numpy array. We then split the data into training and testing sets, and we use the training set to train the model and the testing set to evaluate the model. 

Given that we are training a linear model for regression, we choose the `Root Mean Squared Error (RMSE)` and the `R^2` as our evaluation metrics.

In [ ]:
X = np.concatenate([X_num, X_type, X_region, X_month, X_week], axis=1)
logger.info(f"X shape: {X.shape} .")

train_idx = np.random.choice(X.shape[0], int(0.8 * X.shape[0]), replace=False)
X_train, y_train = X[train_idx], y[train_idx]
logger.info(f"X_train shape: {X_train.shape} , y_train shape: {y_train.shape} .")

test_mask = np.ones((X.shape[0]), dtype=bool)
test_mask[train_idx] = False
X_test, y_test = X[test_mask], y[test_mask]
logger.info(f"X_test shape: {X_test.shape} , y_test shape: {y_test.shape} .")

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
rmse_ = rmse(y_test, y_preds)
r2_ = r2(y_test, y_preds)
logger.info(f"Model RMSE: {rmse_:,.2f}, R^2: {r2_:,.2f} .")

In [ ]:
print(y_test)

In [ ]:
print(y_preds)